<!-- # CNN autoencoder and Clustering from MTRX data

Use this notebook to load Scienta Omicron Matrix format SPM data and create standardised images for machine learning training and analysis. The code can generate both JPG image data, useful for manually checking the data, and windowed numpy data that can be loaded into ML models. 

The notebook then creates an autoencoder for training on a large dataset, followed by KMEANS clustering. 

**Author**: Steven R. Schofield  
**Created**: November, 2024 -->

# CASTEP - Si 001 surface
## Steven R. Schofield (Universtiy College London) May 2025

### Determine appropriate paths whether we are working on macbook or the cluster

In [1]:
import sys
from pathlib import Path

# Define candidate paths using Path objects
module_path_list = [
    Path('/Users/steven/academic-iCloud/Python/modules'),
    Path('/hpc/srs/Python/modules')
]

data_path_list = [
    Path('/Users/steven/academic-iCloud/Calculations/castep/'),
    Path('/hpc/srs/castep')
]

# Resolve actual paths
module_path = next((p for p in module_path_list if p.exists()), None)
data_path = next((p for p in data_path_list if p.exists()), None)

# Check and report missing paths
if module_path is None:
    print("Error: Could not locate a valid module path.")
if data_path is None:
    print("Error: Could not locate a valid data path.")

if module_path is None or data_path is None:
    sys.exit(1)

# Add module_path to sys.path if needed
if str(module_path) not in sys.path:
    sys.path.insert(0, str(module_path))

# Print resolved paths
print(f"module_path = {module_path}")
print(f"data_path = {data_path}")

module_path = /Users/steven/academic-iCloud/Python/modules
data_path = /Users/steven/academic-iCloud/Calculations/castep


### Import modules

In [2]:
# # Ensure modules are reloaded 
%load_ext autoreload
%autoreload 2

# Import standard modules
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt

# Import custom module
import SRSCALCUTILS.castep_tools as ct

from IPython.display import display

### Paths for the calculation

In [3]:
# General
job_folder = 'work-2025/si001_surf'
job_path = data_path / job_folder
verbose = False             # Set this True to print out more information

# Print resolved paths
print(f"job_path = {job_path}")

# If it doesn’t exist, create it (including any missing parent folders)
if not os.path.isdir(job_path):
    os.makedirs(job_path, exist_ok=True)

# Now safely switch into it
os.chdir(job_path)

ct.delete_all_files_in_cwd(force=True)

job_path = /Users/steven/academic-iCloud/Calculations/castep/work-2025/si001_surf
Done. 5 file(s) deleted.


### Base parameters of calculation and unit cell

In [4]:
# Set up the param and cell files for the calculation
filename = "si001_surf"
title = "silicon 001 surface tests"

params = {'task' : 'geomopt',
          'xc_functional'           : 'PBE',
          'cut_off_energy'          : 400,
          'spin_polarised'          : 'false',
          'write_cell_structure'    : 'true',
          'charge'                  : 0,
          'nextra_bands'            : 8,
          'geom_energy_tol'         : 0.00005,          # default: 0.00005 eV 
          'geom_disp_tol'           : 0.002,            # default: 0.002 Ang.
          'geom_max_iter'           : 100,              # default: 30
          'geom_convergence_win'    : 2,                # default: 2
          'max_scf_cycles'          : 300               # default: 30
          }              

lattice_cart = np.array([
            [3.8641976,     0.0,     0.0],
            [0.0,     3.8641976,     0.0],
            [0.0,     0.0,     5.4648012]
        ])

positions_frac = np.array([
            ['Si', 0.0,  0.0,  0.0],
            ['Si', 0.5,  0.0,  0.25],
            ['Si', 0.5,  0.5,  0.5],
            ['Si', 0.0,  0.5,  0.75],
        ])

cell_constraints = np.array([
            [0,  0,  0],
            [0,  0,  0]
        ])

kpoints_mp_grid = np.array([2, 2, 2])

fix_all_ions = False
symmetry_generate=False
symmetry_tol = 0.01


### Build supercell

In [5]:
n = [4,4,1] 
positions_frac, lattice_cart = ct.create_supercell_from_fractional_coords(positions_frac,lattice_cart,n)
positions_frac = ct.sort_positions_frac(positions_frac)

### Atom selection

In [6]:
criteria = ('min','min','max')
selected_frac, selected_cart = ct.select_atom_by_conditions(positions_frac, lattice_cart, criteria)
reference_position = selected_cart[1:]

In [7]:
# atom_number = 1
# selected_frac, selected_cart = ct.select_atom_by_number(positions_frac, lattice_cart, atom_number, element=None)
# reference_position = selected_cart[1:]

### Select atom plane

In [8]:
# import numpy as np

# def selected_toggle_by_periodicity(labelled_positions_frac, n=(2,1,1)):
#     """
#     Toggle True→False every n[i] atoms along axis i within each constant-(other-axes)
#     slice.  No phase offsets.

#     Parameters
#     ----------
#     labelled_positions_frac : array-like of shape (N,6)
#       Rows = [index, is_selected (bool), element, fx, fy, fz]
#     n : tuple of ints (nx,ny,nz)
#       Periodicity along x,y,z. Axes with n[i]==1 are skipped.

#     Returns
#     -------
#     out, out_inverse : np.ndarray of shape (N,6), dtype object
#       - out: toggled version
#       - out_inverse: inverse of out but only among rows originally True
#     """
#     arr = np.asarray(labelled_positions_frac, dtype=object)
#     orig_flags = arr[:,1].astype(bool)
#     fx = arr[:,3].astype(float)
#     fy = arr[:,4].astype(float)
#     fz = arr[:,5].astype(float)
#     coords = [fx, fy, fz]

#     # compute integer grid indices for each axis
#     idxs = [np.searchsorted(np.unique(coords[i]), coords[i]) for i in range(3)]
#     nx, ny, nz = n

#     def _toggle_along_axis(i, flags):
#         Ni = n[i]
#         if Ni <= 1:
#             return flags
#         # the two other axes
#         other = [ax for ax in (0,1,2) if ax != i]
#         u0 = np.unique(idxs[other[0]])
#         u1 = np.unique(idxs[other[1]])

#         new_flags = flags.copy()
#         for g0 in u0:
#             for g1 in u1:
#                 # rows in this slice that were originally True
#                 mask = (idxs[other[0]] == g0) & (idxs[other[1]] == g1) & orig_flags
#                 rows = np.where(mask)[0]
#                 if rows.size == 0:
#                     continue
#                 # sort by axis-i coordinate
#                 order = np.argsort(coords[i][rows])
#                 for pos, r in enumerate(rows[order]):
#                     # flip every Ni-th in this slice
#                     if pos % Ni == 0:
#                         new_flags[r] = False
#         return new_flags

#     # apply toggles axis by axis
#     flags = orig_flags.copy()
#     for axis in (0,1,2):
#         flags = _toggle_along_axis(axis, flags)

#     # build the “out” array
#     out = arr.copy()
#     out[:,1] = flags.tolist()

#     # build the inverse—only flipping among originally True rows
#     inv_flags = orig_flags & (~flags)
#     out_inv = arr.copy()
#     out_inv[:,1] = inv_flags.tolist()

#     return out, out_inv


In [9]:
plane = (0,0,1)
labelled_positions_frac = ct.select_atoms_by_plane(
    positions_frac,
    lattice_cart,
    plane,
    reference_position,
    tolerance=1
)

In [10]:
labelled_positions_frac

[[1, True, 'Si', 0.75, 0.875, 0.75],
 [2, True, 'Si', 0.5, 0.875, 0.75],
 [3, True, 'Si', 0.25, 0.875, 0.75],
 [4, True, 'Si', 0.0, 0.875, 0.75],
 [5, True, 'Si', 0.75, 0.625, 0.75],
 [6, True, 'Si', 0.5, 0.625, 0.75],
 [7, True, 'Si', 0.25, 0.625, 0.75],
 [8, True, 'Si', 0.0, 0.625, 0.75],
 [9, True, 'Si', 0.75, 0.375, 0.75],
 [10, True, 'Si', 0.5, 0.375, 0.75],
 [11, True, 'Si', 0.25, 0.375, 0.75],
 [12, True, 'Si', 0.0, 0.375, 0.75],
 [13, True, 'Si', 0.75, 0.125, 0.75],
 [14, True, 'Si', 0.5, 0.125, 0.75],
 [15, True, 'Si', 0.25, 0.125, 0.75],
 [16, True, 'Si', 0.0, 0.125, 0.75],
 [17, False, 'Si', 0.875, 0.875, 0.5],
 [18, False, 'Si', 0.625, 0.875, 0.5],
 [19, False, 'Si', 0.375, 0.875, 0.5],
 [20, False, 'Si', 0.125, 0.875, 0.5],
 [21, False, 'Si', 0.875, 0.625, 0.5],
 [22, False, 'Si', 0.625, 0.625, 0.5],
 [23, False, 'Si', 0.375, 0.625, 0.5],
 [24, False, 'Si', 0.125, 0.625, 0.5],
 [25, False, 'Si', 0.875, 0.375, 0.5],
 [26, False, 'Si', 0.625, 0.375, 0.5],
 [27, False, 'Si', 

In [25]:
labelled_positions_frac_dimer1, labelled_positions_frac_dimer2 = toggle_plane_selection(
    labelled_positions_frac,
    fast='y',
    slow='x',
    alternate=True
)

filename="dimer1"
positions_frac = ct.selected_replace(labelled_positions_frac_dimer1, element="Si", replacewith="O", return_labelled=False)
positions_cart = ct.frac_to_cart(lattice_cart, positions_frac)
xyz = ct.write_xyz(positions_cart, path=job_path, filename=filename, comment='Converted by SRSCALCUTILS')

filename="dimer2"
positions_frac = ct.selected_replace(labelled_positions_frac_dimer2, element="Si", replacewith="O", return_labelled=False)
positions_cart = ct.frac_to_cart(lattice_cart, positions_frac)
xyz = ct.write_xyz(positions_cart, path=job_path, filename=filename, comment='Converted by SRSCALCUTILS')




### Add hydrogen termination to cell bottom 

In [ ]:
# # First select the atoms on the bottom layer 
# criteria = ('min','min','min')
# selected_frac, selected_cart = ct.select_atom_by_conditions(positions_frac, lattice_cart, criteria)
# reference_position = selected_cart[1:]
# plane = (0,0,1)
# labelled_positions_frac = ct.select_atoms_by_plane(
#     positions_frac,
#     lattice_cart,
#     plane,
#     reference_position,
#     tolerance=1
# )
# # Next calculate the relative bond vectors for the two hydrogen atoms (per Si)
# phi = np.deg2rad(90)    
# si_h_bond_length = 1.5                         
# theta_h_1 = np.deg2rad(180) - np.arccos(-1/3) / 2                
# theta_h_2 = np.deg2rad(180) + np.arccos(-1/3) / 2            
# bond_h_1  = ct.bond_vector_from_spherical(theta_h_1, phi, si_h_bond_length)                   
# bond_h_2  = ct.bond_vector_from_spherical(theta_h_2, phi, si_h_bond_length)

# # Now add the hydrogen atoms to the silicon atoms selected above using the calculated bond vectors
# positions_frac, lattice_cart = ct.add_atoms_to_positions_frac(
#                                     labelled_positions_frac,
#                                     lattice_cart,
#                                     [bond_h_1,bond_h_2],
#                                     extend_unit_cell=(0, 0, 1),
#                                     atom="H")
# positions_frac = ct.sort_positions_frac(positions_frac)

### Dimerise surface

In [ ]:
# # First select the atoms on the bottom layer 
# axis = 'z'
# criteria = "maximum"
# ref_value, _ = ct.find_plane_value(positions_frac, lattice_cart, axis, criteria)
# tolerance = 0.5
# labelled_positions_frac = ct.select_atoms_by_plane(positions_frac, lattice_cart, axis, ref_value,
#                           tolerance=tolerance, include=None, exclude=None)

# # Dimerise surface
# dimer_direction = "y"
# displacement_direction = "y"
# displacement = 0.6
# start_phase='+'
# alternate=False
# positions_frac = dimerise_displacement(labelled_positions_frac,
#     lattice_cart,
#     dimer_direction = dimer_direction,
#     displacement_direction = displacement_direction,
#     displacement = displacement,
#     start_phase=start_phase,
#     alternate=alternate,
#     wrap_axes=(1,1,1)
# )

# # labelled_positions_frac = ct.select_atoms_by_plane(positions_frac, lattice_cart, axis, ref_value,
# #                           tolerance=tolerance, include=None, exclude=None)

# # displacement = 0.3
# # displacement_direction = "z"
# # alternate=True
# # positions_frac = dimerise_displacement(labelled_positions_frac,
# #     lattice_cart,
# #     dimer_direction = dimer_direction,
# #     displacement_direction = displacement_direction,
# #     displacement = displacement,
# #     start_phase=start_phase,
# #     alternate=alternate,
# #     wrap_axes=(1,1,1)
# # )

### Add hydrogen to the top

In [ ]:
# # First select the atoms on the bottom layer 
# axis = 'z'
# criteria = "maximum"
# ref_value, _ = ct.find_plane_value(positions_frac, lattice_cart, axis, criteria)
# tolerance = 0.5
# labelled_positions_frac = ct.select_atoms_by_plane(positions_frac, lattice_cart, axis, ref_value,
#                           tolerance=tolerance, include=None, exclude=None)

# # Next calculate the relative bond vectors for the two hydrogen atoms (per Si)
# phi = np.deg2rad(90)    
# si_h_bond_length = 1.5                         
# theta_h_1 = np.arccos(-1/3) / 2                
# theta_h_2 = np.arccos(-1/3) / 2            
# bond_h_1  = ct.bond_vector_from_spherical(theta_h_1, phi, si_h_bond_length)                   
# bond_h_2  = ct.bond_vector_from_spherical(theta_h_2, phi, si_h_bond_length)

# # Now add the hydrogen atoms to the silicon atoms selected above using the calculated bond vectors
# positions_frac, lattice_cart = ct.add_atoms_to_positions_frac(
#                                     labelled_positions_frac,
#                                     lattice_cart,
#                                     [bond_h_1,bond_h_2],
#                                     extend_unit_cell=(0, 0, 1),
#                                     atom="H")
# positions_frac = ct.sort_positions_frac(positions_frac)

### Add vacuum spacing

In [ ]:
vac = 6
positions_frac, lattice_cart = ct.create_vacuum_spacing(positions_frac, lattice_cart, vac)

### Add ionic contraints

In [ ]:
# # Create fixed ion constraints
# conditions = "z < 2.5"
# ionic_constraints = ct.select_atoms_by_region(positions_frac, lattice_cart, conditions)

In [ ]:
positions_frac

In [ ]:
filename = f"si001_test"
castep_path = job_path / f"{filename}.castep"

print('=' * 80 + f'\nCASTEP file: {castep_path}\n' + '=' * 80 + '\n')   

param_filename = ct.write_param_file(
    params,
    title = title,
    filename=filename,
    path=job_path,
    display_file=True
    )

cell_filename = ct.write_cell_file(
    title = title,
    path=job_path,
    filename=filename,
    lattice_cart=lattice_cart,
    positions_frac=positions_frac,
    cell_constraints=cell_constraints,
#    ionic_constraints=ionic_constraints,
    fix_all_ions=fix_all_ions,
    symmetry_generate=symmetry_generate,
    symmetry_tol = symmetry_tol,
    kpoints_mp_grid=kpoints_mp_grid,
    display_file=True
)

# Convert to cart and write xyz
positions_cart = ct.frac_to_cart(lattice_cart, positions_frac)
xyz = ct.write_xyz(positions_cart, path=job_path, filename=filename, comment='Converted by SRSCALCUTILS')


In [ ]:
# if str(module_path) == '/hpc/srs/Python/modules':
#     !bash -l -c "mpirun -np 62 castep.mpi {filename}"
# else:
#     !zsh -l -c "castepmpi {filename}"


In [ ]:

# unit_cell, a, b, c, alpha, beta, gamma = ct.get_final_lattice_parameters(castep_path)
# energy_optimisation = ct.get_LBFGS_energies(castep_path)
# energies = [val for _, val in energy_optimisation]
# energy = energies[-1]

# print('Optimised energy {} eV'.format(energy))
# print('Optimised lattice constants a,b = {} Ang., c = {} Ang.'.format(a,c))
# print('Unit cell:')
# for line in unit_cell:
#     print(' '*2,np.abs(line))
